In [ ]:
%pip install datasets

In [1]:
import json
import pandas as pd
import os
from sklearn.dummy import DummyClassifier
from datasets import load_dataset

/home/tajak/miniconda3/envs/emma_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Load the train dataset - we use the COPA-HR one, but all COPA versions have the same true labels, so it can be used for all
train_df = pd.read_json("../../datasets/copa-hr-train.jsonl", lines=True)
train_df["text"] = train_df["premise"] + " Hypothesis 0: " + train_df["choice1"] + " Hypothesis 1: " + train_df["choice2"] + "Which hypothesis is more plausible? "
train_df.head(2)

,premise,choice1,choice2,question,label,idx,changed,text
0,Moje je tijelo bacalo sjenu na travu.,Sunce je izlazilo.,Trava je bila pokošena.,cause,0,0,False,Moje je tijelo bacalo sjenu na travu. Hypothes...
1,Žena je tolerirala problematično ponašanje svo...,Žena je znala da njezin prijatelj prolazi kroz...,Žena je imala osjećaj da njezin prijatelj isko...,cause,0,1,False,Žena je tolerirala problematično ponašanje svo...


In [3]:
# Load the test datasets

# Load the test datasets
test_en = pd.read_json("../../datasets/copa-en-test.jsonl", lines=True)
test_sl = pd.read_json("../../datasets/copa-sl-test.jsonl", lines=True)
test_hr_ckm = pd.read_json("../../datasets/copa-hr-ckm-test.jsonl", lines=True)
test_hr = pd.read_json("../../datasets/copa-hr-test.jsonl", lines=True)
test_mk = pd.read_json("../../datasets/copa-mk-test.jsonl", lines=True)
test_sl_cer = pd.read_json("../../datasets/copa-sl-cer-test.jsonl", lines=True)
test_sr = pd.read_json("../../datasets/copa-sr-test.jsonl", lines=True)
test_sr_tor = pd.read_json("../../datasets/copa-sr-tor-test.jsonl", lines=True)

print(test_en.shape, test_sl.shape, test_hr.shape, test_hr_ckm.shape, test_mk.shape, test_sl_cer.shape, test_sr.shape, test_sr_tor.shape)

(500, 5) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5) (500, 7) (500, 5)


In [4]:
display(test_en.head(2))
display(test_hr.head(2))
display(test_hr_ckm.head(2))

,premise,choice1,choice2,question,idx
0,The item was packaged in bubble wrap.,It was fragile.,It was small.,cause,0
1,I emptied my pockets.,I retrieved a ticket stub.,I found a weapon.,effect,1


,premise,choice1,choice2,question,label,idx,changed
0,Predmet je bio zapakiran u omot s mjehurićima.,Bio je lomljiv.,Bio je malen.,cause,0,0,False
1,Ispraznio sam džepove.,Pronašao sam odrezak karte.,Pronašao sam oružje.,effect,0,1,False


,premise,choice1,choice2,question,idx
0,Ta stvar je bila zapakirana va najlon z mehurići.,Ga je bilo lahko zlomit.,Je bi minji.,cause,0
1,San mu sprazni žepi.,San naša kušćić harti za nekamor.,San naša oružje.,effect,1


In [13]:
def dummy(test_df_name):
    # Create X_train and Y_train parts, used for sci kit learning
 
    # List of texts in training split
    X_train = list(train_df.text)
    # List of labels in training split
    Y_train = list(train_df.label)

    test_df_dict = {"copa-en": test_en,
                    "copa-sl": test_sl,
                    "copa-hr": test_hr,
                    "copa-hr-ckm": test_hr_ckm,
                    "copa-mk": test_mk,
                    "copa-sl-cer": test_sl_cer,
                    "copa-sr": test_sr,
                    "copa-sr-tor": test_sr_tor,
	}

    test_df = test_df_dict[test_df_name]

    # List of texts in test split
    test_df["text"] = test_df["premise"] + " Hypothesis 0: " + test_df["choice1"] + " Hypothesis 1: " + test_df["choice2"] + "Which hypothesis is more plausible? "
    X_test = list(test_df.text)
    # List of labels in test split
    y_true_file = open("../../datasets/test_labels.txt", "r").readlines()
    Y_test = [int(x.replace("\n", "")) for x in y_true_file]

    print(len(X_train), len(Y_train), len(X_test), len(Y_test))

    # Create a list of labels
    labels = [0, 1]
    print("Labels: {}".format(labels))

    for strategy in ["stratified", "most_frequent"]:
        model = f"dummy-{strategy}"

        dummy_mf = DummyClassifier(strategy=strategy)

        # Train the model
        dummy_mf.fit(X_train, Y_train)

        #Get the predictions
        y_pred_mf = dummy_mf.predict(X_test)

        y_pred = [int(x) for x in y_pred_mf]

        # Create a json with results
        current_results = {
            "system": model,
            "predictions": [
                {
                "train": "COPA-HR-train",
                "test": "{}".format(test_df_name),
                "predictions": y_pred,
                }
            ],
            }

        # Save the results as a new json
        with open("submissions/submission-{}-{}.json".format(model, test_df_name), "w") as file:
            json.dump(current_results, file)

        print("Classification with {} on {} finished.".format(model, test_df_name))


In [11]:
tests = ["copa-en","copa-sl", "copa-hr", "copa-hr-ckm", "copa-mk", "copa-sl-cer", "copa-sr", "copa-sr-tor"]

In [14]:
for test in tests:
	dummy(test)


400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-en finished.
Classification with dummy-most_frequent on copa-en finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-sl finished.
Classification with dummy-most_frequent on copa-sl finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-hr finished.
Classification with dummy-most_frequent on copa-hr finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-hr-ckm finished.
Classification with dummy-most_frequent on copa-hr-ckm finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-mk finished.
Classification with dummy-most_frequent on copa-mk finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-sl-cer finished.
Classification with dummy-most_frequent on copa-sl-cer finished.
400 400 500 500
Labels: [0, 1]
Classification with dummy-stratified on copa-sr finis